In [1]:
import numpy as np

In [2]:
### Important functions

def sigmoid_exact(x):
    return 1 / (1 + np.exp(-x))

# using taylor series
def sigmoid_approximation(x):
    return (1/2) + (x/4) - (x**3 / 48) + (x**5 / 480)

for lil_number in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    
    print("\nInput:" + str(lil_number))
    print("Exact Sigmoid:" + str(sigmoid_exact(lil_number)))
    print("Approx Sigmoid:" + str(sigmoid_approximation(lil_number)))
    



Input:0.1
Exact Sigmoid:0.52497918747894
Approx Sigmoid:0.5249791874999999

Input:0.2
Exact Sigmoid:0.549833997312478
Approx Sigmoid:0.549834

Input:0.3
Exact Sigmoid:0.574442516811659
Approx Sigmoid:0.5744425624999999

Input:0.4
Exact Sigmoid:0.598687660112452
Approx Sigmoid:0.598688

Input:0.5
Exact Sigmoid:0.6224593312018546
Approx Sigmoid:0.6224609375000001

Input:0.6
Exact Sigmoid:0.6456563062257954
Approx Sigmoid:0.6456620000000001

Input:0.7
Exact Sigmoid:0.6681877721681662
Approx Sigmoid:0.6682043125000001

Input:0.8
Exact Sigmoid:0.6899744811276125
Approx Sigmoid:0.690016

Input:0.9
Exact Sigmoid:0.7109495026250039
Approx Sigmoid:0.7110426875

Input:1.0
Exact Sigmoid:0.7310585786300049
Approx Sigmoid:0.73125


### Using Efficient Integer Vector Homomorphic Encryption

* S: matrix that represents secret key
* M: public key used to encrypt and perofrm oepations
* c: ciphertext
* x: plaintext
* w: weighted scalar, used to tune signal/noise ratio. Making the signal bigger makes it less susceptible to noise at any given operation. However, making it too big increases the likelihood of corrupting the data entirely
* E/e: noise


$Sc = wx + e$

and

$x = \lceil \frac{Sc}{w} \rfloor$

* The general philosophy of Homomorphic Encryption techniques is to introduce just enough noise that the original message is hard to get back without the secret key, but small enough amount of noise that it amounts to a rounding error when you DO have the secret key
* Encryption is about generating c so that this relationship is true
* If S is a random matrix, then c will be hard to decrypt
* The simpler, non-symmetric way of generating an encyrption key is just fidn the inverse of the secret key

In [3]:
def generate_key(w,m,n):
    # we want max(S) < w
    S = (np.random.rand(m,n) * w / (2**16))
    return S


In [4]:
def encrypt(x,S,m,n,w):
    assert len(x) == len(S)
    
    e = (np.random.rand(m))
    c = np.linalg.inv(S).dot((w*x) + e)
    return c

In [5]:
def decrypt(c,S,w):
    return (S.dot(c) / w).astype('int')

In [6]:
# convert ciphertext c from integer to binary representation

def get_c_star(c,m,l):
    c_star = np.zeros(l*m, dtype='int')
    for i in range(m):
        b = np.array(list(np.binary_repr(np.abs(c[i]))),dtype='int')
        if (c[i] < 0):
            b *= -1
        c_star[(i*l) + (l-len(b)): (i+1) * l ] += b
    return c_star

In [7]:
# convert S into 'gadget' form

def get_S_star(S,m,n,l):
    S_star = list()
    for i in range(l):
        S_star.append(S*2**(l-i-1))
    S_star = np.array(S_star).transpose(1,2,0).rehape(m,n*l)
    return S_star

In [8]:
### Let's try an example

x = np.array([0,1,2,5])
m = len(x)
n = m
w = 16
S = generate_key(w,m,n)

In [9]:
c = encrypt(x,S,m,n,w)
decrypt(c,S,w)

array([0, 1, 2, 5])

In [10]:
decrypt(c+c,S,w)

array([ 0,  2,  4, 10])

In [11]:
decrypt(c*10,S,w)

array([ 0, 10, 20, 50])

### Optimizing Encryption

* The current encryption strategy doesn't make sense

$Sc = wx + e$

and

$x = \lceil \frac{Sc}{w} \rfloor$

if the secret key $S$ is the identity matrix, then the cyphertext c is just a linearly weighted version of the message x

* Instead of explicitly allocating a self-standing "Public Key" and "Private Key" the authors propose a "Key Swtiching" technique where they can swap out one Private Key S for another S'

* Basically it involves finding a matrix M that can perform the transfomration

* Since M has the ability to converg a message from being unencrypted (secret key of identity matrix) to being encrypted (secret key that's random and difficult to guess) M comes our public key

i.e.

* Because M performs the role of a "one way encryption" we call it the "public key: 

### Notes to self

* To-do: read the paper again and understand key switching (diagram it out)
* To-do: implement addition, linear transform, and weighted inner product

In [ ]:
# Let's remind ourselves of the dimensions
# x is n-dimensional
# c is m-dimensional
# Sc = wx + e
# S is n x m

def get_c_star(c, m, l):
    # the return type is length m * l
    c_star = np.zeros(m*l, dtype='int')
    for i in range(m):
        # convert c[i] to binary
        # you can use np.binary_rep(np.abs(c[i])), which returns a string type
        b = np.array(list(np.binary_rep(np.abs(c[i]))))
        if c[i] < 0:
            b *= -1
        c_star[i*l + (l-len(b)): (i+1)*l] += b
    return c_star

"""
Need to test this function
"""
def get_S_star(S, m, n, L):
    S_star = np.zeros(n, m*l, dtype='int')
    
    # should make a base repr, to do elementwise mutliplication with
    base_repr = np.zeros(l, dtype='int')
    for i in range(m):
        base_repr[m-i-1] = 2**i
        
    # for i in range(3) --> [0,1,2]
    # idx 0: (3 - 2 - 1) 4
    # idx 1: (3 - 1 - 1) 2
    # idx 2: (3 - 0 - 1) 1
    # [4,2,1]
    
    for i in range(n):
        for j in range(m):
            # expand S[i,j]
            b_tmp = base_repr * S[i,j]
            S_star[i, j*l: (j+1)*l] += b_tmp
    
    return S_star

def key_switch(S_in,c_in,m,n): # outputs S_out, c_out, M
    # want abs(c) < 2^l
    # thus, l > log_2 abs(c)
    l = int(np.ceil(np.log2(np.max(np.abs(c)))))
    
    # step 1, expand ciphertext and key into bit representation and gadget representation
    c_star = get_c_star(c,m,l)
    S_star = get_S_star(S, m, n, L)
    
    # The remainder is easy
    # Just generate random matrices A, E and invert to get S_prime, c_prime
    
    